# Grupo 6
## Integrantes 
- Pablo Sarzosa
- Elisa Herrera
- Doménica Rodríguez 
- Andrés Albarracín

# 4.4.4. Regresión polinomial

Ahora podemos explorar estos conceptos de forma interactiva ajustando polinomios a los datos.

In [1]:
use strict;
use warnings;
use List::Util qw(min shuffle);
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use AI::MXNet qw(nd);
use d2l;
use d2l::Animator;
use d2l::Accumulator;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly qw(show_plot);

use constant np => 'numperl'; #Tener en cuenta el uso de numperl
use constant False => 'False';
use constant True => 'True';
use constant None => 'None';




# 4.4.4.1. Generación del conjunto de datos
Primero necesitamos datos. Dado x
 , usaremos el siguiente polinomio cúbico para generar las etiquetas en los datos de entrenamiento y prueba:

$$ y=5+1.2x−3.4\frac{x^2}{2!}+5.6*\frac{x^3}{3!} + ϵ  where  ϵ∼N(0,0.1^2) 

$$ 
El término de ruido $ ϵ $
 sigue una distribución normal con media 0 y desviación estándar 0.1. Para la optimización, normalmente queremos evitar valores muy grandes de gradientes o pérdidas. Esta es la razón por la cual las características se reescalan de xi
 a xii!
 . Nos permite evitar valores muy grandes para exponentes grandes i
 . Sintetizaremos 100 muestras cada una para el conjunto de entrenamiento y el conjunto de prueba.

In [2]:
my $maxDegree = 20;
my $nTrain = 100;
my $nTest = 100;
my $trueW = mx->nd->zeros([$maxDegree]);
my $array = mx->nd->array([5, 1.2, -3.4, 5.6]);

$trueW -> slice([0,3]) += $array->slice([0,3]);
my $features = mx->nd->random_normal(shape=>[$nTrain + $nTest, 1]);


$features = mx -> nd -> shuffle($features);
my $aux = mx -> nd -> array([0 .. $maxDegree - 1]);
my $polyFeatures = $features ** $aux -> reshape([1,-1]);
for my $i (0 .. $maxDegree - 1) {
    for my $j (0 .. $maxDegree*10 - 1) {#$features->len
        $polyFeatures->[$j][$i] /= mx->nd->gamma(mx->nd->array([$i + 1]));
    }
}


my $labels = mx->nd->dot($polyFeatures, $trueW);
$labels += mx->nd->randomNormal(scale => 0.1, shape => $labels->shape);

Error: Can't locate object method "randomNormal" via package "AI::MXNet::NDArray" at reply input line 22.


Nuevamente, los monomios almacenados en poly_featuresson reescalados por la función gamma, donde  $Γ(𝑛)=(𝑛−1)!$  . Eche un vistazo a las primeras 2 muestras del conjunto de datos generado. El valor 1 es técnicamente una característica, es decir, la característica constante correspondiente al sesgo.

In [3]:
print $features -> slice([0,1]) -> aspdl, $polyFeatures -> slice([0,1]) -> slice([0,1]) -> aspdl, $labels -> slice([0,1]) -> aspdl;


[
 [1.50948]
 [1.96766]
]

[
 [      1 1.50948 2.27852 3.43936 5.19163 7.83664 11.8292 17.8559  26.953 40.6849 61.4129 92.7012  139.93 211.221 318.833 481.271 726.466 1096.58 1655.26 2498.58]
 [      1 1.96766 3.87169 7.61818   14.99 29.4952 58.0366 114.196   224.7 442.133 869.968  1711.8 3368.25 6627.57 13040.8 25659.9   50490 99347.2  195482  384642]
]
[18.3248 36.8592]

1

In [4]:
sub train_epoch_ch3{
    my ($net, $train_iter, $loss, $updater, $batch_size) = @_;
    my $metric = Accumulator->new(3);
    if ($updater->isa('Gluon::Trainer')){
        $updater->step($batch_size, 1);
    }  
    my ($X, $y, $y_hat, $l);
    while(defined(my $batch = <$train_iter>)){
        # Compute gradients and update parameters
        $X = $batch->[0];
        $y = $batch->[1]->astype('float32'); 
        autograd->record(sub {$y_hat = $net->($X);
                                $l = $loss->($y_hat, $y);
                                });
        $l->backward();
        if (ref($updater) eq 'AI::MXNet::Gluon::Trainer'){
            $updater->step($batch_size);
        }else{
            $updater->($batch_size);
        }
        $metric->add([ $l->sum->asscalar, accuracy($y_hat, $y), $y->size ]);
    }
    # Return training loss and training accuracy
    return ($metric->getitem(0) / $metric->getitem(2), $metric->getitem(1) / $metric->getitem(2));
}

# 4.4.4.2. Entrenamiento y prueba del modelo
Primero implementemos una función para evaluar la pérdida en un conjunto de datos dado.

In [5]:
sub evaluate_loss{
    my($net, $dataIter, $loss ) = @_;
    my $metric = Accumulator->new(2);
    my ($X , $y ,$l);
    while(defined(my $batch = <$dataIter>)){ 
        $X = $batch->[0];
        $y = $batch->[1]->astype('float32');
        my $l = $loss->($net->($X), $y);
        $metric->add([ $l->sum(), $l->size ]);
    }
    return ($metric->getitem(0) / $metric->getitem(1));
}

Ahora defina la función de entrenamiento.

In [10]:
sub train {
    my ($trainFeatures, $testFeatures, $trainLabels, $testLabels, $numEpochs) = @_;
    $numEpochs = 400;
    my $loss = mx->gluon->loss->L2Loss();
    my $net = mx->gluon->nn->Sequential();
    $net->add(mx->gluon->nn->Dense(1, use_bias => 0));
    $net->initialize();
    my $batchSize = min(10, ($trainLabels->shape)[0]);
    
    my $trainIter = d2l->load_array([$trainFeatures, $trainLabels], $batchSize);
    my $testIter = d2l->load_array([$testFeatures, $testLabels], $batchSize, 0);
    my $trainer = mx->gluon->Trainer($net->collect_params(), 'sgd', {'learning_rate' => 0.01});
    my $animator = Animator->new(xlabel => 'epoch', ylabel => 'loss', xlim => [1, $numEpochs], ylim => [1e-3, 1e2], legend => ['train', 'test']);
    
    for (my $epoch = 0; $epoch < $numEpochs; $epoch++) {
        #train_epoch_ch3($net, $train_iter, $loss, $trainer, $batch_size);
        if ($epoch == 0 || ($epoch + 1) % 20 == 0) {
            $animator->add($epoch + 1, [evaluate_loss($net, $trainIter, $loss), evaluate_loss($net, $testIter, $loss)]);
        }
    }
    #show_plot($animator);
    print "weight:", $net->[0]->weight->data->aspdl, "\n";
}

my $params = my $net->collect_params('.*weight|.*bias');
print dump $params->[1]->{_data}->[0]->asarray;

Warning: Subroutine train redefined at reply input line 1.


Error: Can't call method "collect_params" on an undefined value at reply input line 25.


# 4.4.4.3. Ajuste de función polinomial de tercer orden (normal)
Comenzaremos usando primero una función polinomial de tercer orden, que es del mismo orden que la función de generación de datos. Los resultados muestran que las pérdidas de entrenamiento y prueba de este modelo pueden reducirse de manera efectiva. Los parámetros del modelo aprendido también están cerca de los valores reales  $w=[5,1.2,−3.4,5.6]$
 .

In [7]:
train($polyFeatures->slice([0,$nTrain-1],[0,3])->aspdl,
        $polyFeatures->slice([$nTrain, -1],[0,3])->aspdl,
        $labels->slice([0,$nTrain-1])->aspdl,
        $labels->slice([$nTrain,-1])->aspdl
        );

weight:
[
 [-0.0545475  0.0244653  0.0218861 -0.0255918]
]



1

# 4.4.4.4. Ajuste de función lineal (ajuste inferior)
Echemos otro vistazo al ajuste de funciones lineales. Después del declive en las primeras épocas, se vuelve difícil disminuir aún más la pérdida de entrenamiento de este modelo. Una vez completada la última iteración de época, la pérdida de entrenamiento sigue siendo alta. Cuando se utilizan para ajustar patrones no lineales (como aquí la función polinomial de tercer orden), los modelos lineales tienden a no ajustarse correctamente.

In [8]:
train($polyFeatures->slice([0,$nTrain-1],[0,1])->aspdl,
        $polyFeatures->slice([$nTrain, -1],[0,1])->aspdl,
        $labels->slice([0,$nTrain-1])->aspdl,
        $labels->slice([$nTrain,-1])->aspdl
        );

weight:
[
 [-0.0506544  0.0389684]
]



1

# 4.4.4.5. Ajuste de funciones polinómicas de orden superior (sobreajuste)

Ahora intentemos entrenar el modelo usando un polinomio de grado demasiado alto. Aquí, no hay datos suficientes para saber que los coeficientes de mayor grado deberían tener valores cercanos a cero. Como resultado, nuestro modelo excesivamente complejo es tan susceptible que está siendo influenciado por el ruido en los datos de entrenamiento. Aunque la pérdida de entrenamiento se puede reducir de manera efectiva, la pérdida de prueba sigue siendo mucho mayor. Muestra que el modelo complejo sobreajusta los datos.

In [9]:
train($polyFeatures->slice([0,$nTrain-1],[0,-1])->aspdl,
        $polyFeatures->slice([$nTrain, -1],[0,-1])->aspdl,
        $labels->slice([0,$nTrain-1])->aspdl,
        $labels->slice([$nTrain,-1])->aspdl,
        my $numEpochs=1500
        );

weight:
[
 [ -0.0424785   0.0629399  -0.0183785   0.0227538   0.0449391     -0.0681  -0.0564058   0.0171985   0.0473123   0.0243123  -0.0565462   0.0660723   0.0667043   0.0529471 -0.00438883  0.00134741   0.0667466  -0.0621999   0.0146784 -0.00683771]
]



1

En las secciones subsiguientes, continuaremos analizando los problemas de sobreajuste y los métodos para tratarlos, como la disminución del peso y la deserción.

# 4.4.5. Resumen
* Dado que el error de generalización no se puede estimar en función del error de entrenamiento, simplemente minimizar el error de entrenamiento no significará necesariamente una reducción en el error de generalización. Los modelos de aprendizaje automático deben tener cuidado de protegerse contra el sobreajuste para minimizar el error de generalización.

* Se puede usar un conjunto de validación para la selección del modelo, siempre que no se use con demasiada liberalidad.

* Underfitting significa que un modelo no es capaz de reducir el error de entrenamiento. Cuando el error de entrenamiento es mucho menor que el error de validación, hay sobreajuste.

* Deberíamos elegir un modelo adecuadamente complejo y evitar usar muestras de entrenamiento insuficientes.